In [1]:
# --- 1. 라이브러리 설치 및 임포트 ---

import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.neighbors import BallTree
from datetime import datetime

# sys, os: 파일 경로 조작 및 파이썬 경로 설정
import sys, os

# 프로젝트 루트/src 경로를 PYTHONPATH에 추가하여 Logger 모듈 임포트 준비
src_path = os.path.abspath(os.path.join(os.getcwd(), "../../src"))
sys.path.insert(0, src_path)

# Logger 클래스: 실행 결과를 파일에 기록 및 콘솔 출력
from data.log.logger import Logger

In [2]:

# --- 2. 경로 설정 및 데이터 로드 ---

# 청크 단위 처리를 위한 상수 정의
RADIUS_EARTH_KM = 6371.0
CHUNK_SIZE      = 100_000

# 학습 데이터 파일 경로
RAW_DIR         = '../../data/raw'
OUT_DIR         = '../../data/processed/transportation-features'
TRAIN_RAW       = os.path.join(RAW_DIR, 'train.csv')
TEST_RAW        = os.path.join(RAW_DIR, 'test.csv')
SUBWAY_RAW      = os.path.join(RAW_DIR, 'subway_feature.csv')
BUS_RAW         = os.path.join(RAW_DIR, 'bus_feature.csv')

# 출력 파일 경로 설정
TRAIN_OUT       = os.path.join(OUT_DIR, 'train_transportation_features.csv')
TEST_OUT        = os.path.join(OUT_DIR, 'test_transportation_features.csv')

# Log 파일 경로
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_DIR         = '../../data/logs/transportation-features_logs'
LOG_FILENAME    = f"feature_engineering_transportation_{timestamp}.log"
LOG_PATH        = os.path.join(LOG_DIR, LOG_FILENAME)

# Directory 생성
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

logger = Logger(LOG_PATH)

In [ ]:
# --- 3. 정류장·역 데이터 로드 + BallTree 생성 ---
# — 기능: 지하철·버스 정류장 좌표 로드 및 BallTree 객체 생성
# — 입력: SUBWAY_RAW, BUS_RAW 경로
# — 출력: subway_tree, bus_tree (BallTree)

logger.write("지하철, 버스 정류장 데이터를 로드하고 BallTree를 생성합니다...")
logger.write("데이터를 로드합니다...")

try:
    # — 지하철 위도·경도 컬럼만 읽기
    subway_df = pd.read_csv(SUBWAY_RAW, usecols=['위도','경도'], dtype='float32')
    # — 버스 Y좌표·X좌표 컬럼만 읽기
    bus_df    = pd.read_csv(BUS_RAW,    usecols=['Y좌표','X좌표'], dtype='float32')

    # — 위도·경도(deg) → 라디안(rad) 변환
    subway_rad = np.deg2rad(subway_df[['위도','경도']].values)
    bus_rad    = np.deg2rad(bus_df[['Y좌표','X좌표']].values)

    # — Haversine 거리(metric)로 BallTree 생성
    subway_tree = BallTree(subway_rad, metric='haversine')
    bus_tree    = BallTree(bus_rad,    metric='haversine')

    logger.write("버스, 지하철 데이터 로드 완료.")

except FileNotFoundError as e:
    # — 오류 처리: 파일 미존재 시 경로 확인 안내
    logger.write(f"오류: 파일 로드에 실패했습니다. 경로를 확인해주세요. \n>> {e}", print_error=True)

2025-07-11 16:25:54 | 지하철, 버스 정류장 데이터를 로드하고 BallTree를 생성합니다...
2025-07-11 16:25:54 | 데이터를 로드합니다...
2025-07-11 16:25:54 | 버스, 지하철 데이터 로드 완료.


In [ ]:
# --- 4. 원하는 컬럼 순서 정의 ---
# — 기능: 파생된 교통 관련 피처를 지정한 순서대로 배치하기 위한 컬럼 순서 목록 설정
# — 출력: col_order (리스트 형태)

col_order = [
    '지하철최단거리',
    '반경_1km_지하철역_수',
    '반경_500m_지하철역_수',
    '반경_300m_지하철역_수',
    '버스최단거리',
    '반경_1km_버스정류장_수',
    '반경_500m_버스정류장_수',
    '반경_300m_버스정류장_수',
]

logger.write("컬럼 순서를 정의합니다...")
logger.write(f"컬럼 순서: {col_order}")

In [ ]:
# --- 5. 파생 피처 계산 함수 ---
# — 기능: 좌표 정보로부터 교통 관련 파생 피처(최단거리, 반경 내 역·정류장 개수) 계산

logger.write("파생 피처 계산 함수를 정의합니다...")

# — 입력: df (좌표Y, 좌표X 컬럼 포함된 DataFrame)
# — 출력: out (파생 피처가 포함된 DataFrame)
def compute_transport_features(df):
    # — 좌표(deg) → 라디안(rad) 변환
    coords_rad = np.deg2rad(df[['좌표Y','좌표X']].values)

    # 1) 최단거리 계산
    # — subway_tree, bus_tree를 사용해 각 지점에서 가장 가까운 역/정류장까지 거리 조회
    dist_sub, _ = subway_tree.query(coords_rad, k=1)
    dist_bus,  _ = bus_tree.query(coords_rad, k=1)

    # — 거리(m → km) 환산 후 DataFrame 생성
    out = pd.DataFrame({
        '지하철최단거리': dist_sub.flatten() * RADIUS_EARTH_KM,
        '버스최단거리': dist_bus.flatten() * RADIUS_EARTH_KM,
    }, index=df.index)

    # 2) 반경 내 역·정류장 개수 계산
    # — 반경 목록 정의 (단위: km)
    radii = [(1.0, '1km'), (0.5, '500m'), (0.3, '300m')]
    for km, label in radii:
        # — km → 라디안 반경
        rad = km / RADIUS_EARTH_KM

        # — 반경 내 역 개수 조회
        cnt_sub = subway_tree.query_radius(coords_rad, rad, count_only=True)
        # — 반경 내 정류장 개수 조회
        cnt_bus = bus_tree.query_radius(coords_rad, rad, count_only=True)

        # — 결과를 컬럼으로 추가
        out[f'반경_{label}_지하철역_수']    = cnt_sub
        out[f'반경_{label}_버스정류장_수'] = cnt_bus

    return out

In [ ]:
# --- 5-1) Train 데이터: 청크 단위 처리 및 미리보기 ---
# — 기능: TRAIN_RAW 파일을 CHUNK_SIZE 단위로 읽어와
#         파생 피처를 계산한 뒤 하나의 DataFrame으로 결합
# — 입력: TRAIN_RAW, CHUNK_SIZE, compute_transport_features(), col_order
# — 출력: train_features_df (전체 파생 피처 DataFrame) 및 상위 10개 미리보기

logger.write("Train 데이터 파생 피처를 계산합니다...")

# — 각 청크별로 생성된 파생 피처 DataFrame을 저장할 빈 리스트 초기화
train_parts = []

try:
    # 청크 단위로 아파트명·좌표 컬럼 읽기
    for chunk in pd.read_csv(
            TRAIN_RAW,
            usecols=['아파트명','좌표X','좌표Y'],
            dtype={'아파트명':'string','좌표X':'float32','좌표Y':'float32'},
            chunksize=CHUNK_SIZE
        ):
        # 각 청크에 대해 파생 피처 계산
        feats = compute_transport_features(chunk)
        
        # 아파트명 컬럼을 맨 앞에 삽입
        feats.insert(0, '아파트명', chunk['아파트명'].values)
        
        # 지정된 컬럼 순서로 재배치
        feats = feats[['아파트명'] + col_order]
        train_parts.append(feats)

    # 모든 청크 결합하여 하나의 DataFrame 생성
    train_features_df = pd.concat(train_parts, ignore_index=True)

    logger.write("🎉 Train 데이터 파생 피처 계산 완료.")
    logger.write(f"Train 데이터 상위 10개 행:\n{train_features_df.head(10)}")

except FileNotFoundError as e:
    # — 오류: TRAIN_RAW 파일을 찾을 수 없음
    logger.write(f"❌ 오류: TRAIN_RAW 파일을 찾을 수 없습니다.\n>> {e}", print_error=True)

except Exception as e:
    # — 오류: 파생 피처 처리 중 예외 발생
    logger.write(f"❌ 오류: 파생 피처 처리 중 예외가 발생했습니다.\n>> {e}", print_error=True)

In [ ]:
# --- 5-2) Test 데이터: 전체 처리 및 미리보기 ---
# — 기능: TEST_RAW 파일 전체를 읽어와
#         파생 피처를 계산한 뒤 상위 10개 미리보기
# — 입력: TEST_RAW, compute_transport_features(), col_order
# — 출력: test_features_df (파생 피처 DataFrame) 및 상위 10개 미리보기

try:
    logger.write("Test 데이터 파생 피처를 계산합니다...")
    
    # — 전체 데이터 읽기
    test_df = pd.read_csv(
        TEST_RAW,
        usecols=['아파트명','좌표X','좌표Y'],
        dtype={'아파트명':'string','좌표X':'float32','좌표Y':'float32'}
    )
    
    # — 파생 피처 계산
    test_features_df = compute_transport_features(test_df)
    
    # — 아파트명 컬럼 삽입
    test_features_df.insert(0, '아파트명', test_df['아파트명'].values)
    
    # — 컬럼 순서 재배치
    test_features_df = test_features_df[['아파트명'] + col_order]
    
    # — 완료 로그
    logger.write("🎉 Test 데이터 파생 피처 계산 완료.")
    logger.write(f"Test 데이터 상위 10개 행:\n{test_features_df.head(10)}")

except FileNotFoundError as e:
    # — 오류 처리: TEST_RAW 파일을 찾을 수 없음
    logger.write(f"❌ 오류: TEST_RAW 파일을 찾을 수 없습니다.\n>> {e}", print_error=True)

except Exception as e:
    # — 오류 처리: 파생 피처 처리 중 예외 발생
    logger.write(f"❌ 오류: Test 데이터 파생 피처 처리 중 예외가 발생했습니다.\n>> {e}", print_error=True)

In [ ]:
# --- 6. 결과 저장 ---

# 파생변수 데이터프레임이 정상적으로 생성되었는지 확인 후 저장
try:
    logger.write("파생변수 데이터를 CSV로 저장합니다...")
    
    # 파생변수 DataFrame을 CSV로 저장
    train_features_df.to_csv(TRAIN_OUT, index=False)
    test_features_df.to_csv(TEST_OUT, index=False)

    logger.write(f"🎉 train.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '{TRAIN_OUT}'")
    logger.write(f"🎉 test.csv 파생변수 데이터 저장이 완료되었습니다. 파일 경로: '{TEST_OUT}'")

# 저장 중 오류 발생 시 안내
except Exception as e:
    logger.write(f"\n오류: 파일 저장 중 문제가 발생했습니다. \n>> {e}", print_error=True)